In [1]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

2023-04-07 18:26:31.223559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NUM_COLUMNS = 18
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64

def decode_kafka_item(item):
  message = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(item.key)
  return (message, key)

train_ds = tfio.IODataset.from_kafka("tf.public.susy_train", partition=0, offset=0, servers="kafka:9092")
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

2023-04-07 18:26:34.278824: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-04-07 18:26:34.279129: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2023-04-07 18:26:34.442188: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: bootstrap.servers=kafka:9092
2023-04-07 18:26:34.971617: I tensorflow_io/core/kernels/kafka_kernels.cc:349] Kafka tail: 5000


In [3]:
OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(NUM_COLUMNS,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [4]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10


2023-04-07 18:26:47.187550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]
2023-04-07 18:26:47.188234: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [1]
	 [[{{node Placeholder/_10}}]]
/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-04-07 18:26:48.056947: I tensorflow_io/c

      1/Unknown - 1s 1s/step - loss: 0.7539 - accuracy: 0.5312

2023-04-07 18:26:48.605635: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 1024


     27/Unknown - 2s 20ms/step - loss: 0.6543 - accuracy: 0.6221

...

47/79 [================>.............] - ETA: 0s - loss: 0.4410 - accuracy: 0.8032

2023-04-07 18:27:12.589087: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 3072


63/79 [======================>.......] - ETA: 0s - loss: 0.4458 - accuracy: 0.7944

2023-04-07 18:27:13.094510: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 4096


79/79 [==============================] - 2s 28ms/step - loss: 0.4516 - accuracy: 0.7902


In [5]:
test_ds = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["tf.public.susy_test"],
    group_id="testcg2",
    servers="kafka:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

def decode_kafka_test_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

test_ds = test_ds.map(decode_kafka_test_item)
test_ds = test_ds.batch(BATCH_SIZE)


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `tf.data.Dataset.take_while(...)


2023-04-07 18:27:31.798471: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: session.timeout.ms=7000
2023-04-07 18:27:31.798582: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: max.poll.interval.ms=8000
2023-04-07 18:27:31.798654: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: auto.offset.reset=earliest
2023-04-07 18:27:31.798731: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: group.id=testcg2
2023-04-07 18:27:31.798770: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: bootstrap.servers=kafka:9092
2023-04-07 18:27:31.798859: I tensorflow_io/core/kernels/kafka_kernels.cc:919] max num of messages per batch: 10000
2023-04-07 18:27:31.798919: I tensorflow_io/core/kernels/kafka_kernels.cc:938] Creating the kafka consumer
2023-04-07 18:27:31.799774: I tensorflow_io/core/kernels/kafka_kernels.cc:945] Subscribing to the kafka topic: tf.public.susy_test


In [6]:
res = model.evaluate(test_ds)
print("test loss, test acc:", res)

2023-04-07 18:27:40.136841: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]
2023-04-07 18:27:40.349601: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Assign partitions
2023-04-07 18:27:40.350376: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2023-04-07 18:27:40.350405: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: tf.public.susy_test[0], OFFSET: -1001 ERROR_CODE: 0
2023-04-07 18:27:40.350417: I tensorflow_io/core/kernels/kafka_kernels.cc:802] REBALANCE: Assigning partitions


     52/Unknown - 1s 2ms/step - loss: 0.4517 - accuracy: 0.7924 

2023-04-07 18:27:40.974612: I tensorflow_io/core/kernels/kafka_kernels.cc:996] EOF reached for all 1 partition(s)


79/79 [==============================] - 11s 129ms/step - loss: 0.4477 - accuracy: 0.7948
test loss, test acc: [0.44765180349349976, 0.7947999835014343]


2023-04-07 18:27:50.986206: E tensorflow_io/core/kernels/kafka_kernels.cc:1001] Local: Timed out
